In [ ]:
#Necessary Libraries
!pip install numpy
!pip install opencv-python
!pip install matplotlib
!pip install -U scikit-learn
!pip install pandas
!pip install seaborn
!pip install time
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure
import pandas as pd

## Loading Dataset

In [ ]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xzvf cifar-10-python.tar.gz

In [ ]:
data_batch_1 = unpickle(r'cifar-10-batches-py/data_batch_1')
print(type(data_batch_1))
print("--------------------------")
print(data_batch_1.keys())
print("--------------------------")
for item in data_batch_1:
    print(item, type(data_batch_1[item]))
print("--------------------------")
print("Labels:", set(data_batch_1['labels']))

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
    return dict

batch_1 = unpickle('cifar-10-batches-py/data_batch_1')
batch_2 = unpickle('cifar-10-batches-py/data_batch_2')
batch_3 = unpickle('cifar-10-batches-py/data_batch_3')
batch_4 = unpickle('cifar-10-batches-py/data_batch_4')
batch_5 = unpickle('cifar-10-batches-py/data_batch_5')
test_batch =unpickle('cifar-10-batches-py/test_batch')

X_train_pixel = np.concatenate([batch_1['data'], batch_2['data'], batch_3['data'], batch_4['data'], batch_5['data']], axis=0)
y_train_pixel = np.concatenate([batch_1['labels'], batch_2['labels'], batch_3['labels'], batch_4['labels'], batch_5['labels']], axis=0)

X_test_pixel = test_batch['data']
y_test_pixel = test_batch['labels']

# Reshape the image data to its original dimensions
X_train_pixel = X_train_pixel.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
X_test_pixel = X_test_pixel.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

print(f'Training Samples: {len(X_train_pixel)}')
print(f'Testing Samples: {len(X_test_pixel)}')

In [ ]:
print(X_train_pixel[0].shape)
print(X_test_pixel[0].shape)

## Preprocessing

In [ ]:
def rgb2gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# Converting to grayscale
X_training_gray = [ rgb2gray(np.array(X_train_pixel[i])) for i in range(50000)]
X_testing_gray  = [ rgb2gray(np.array(X_test_pixel[i])) for i in range(10000)]

In [ ]:
%matplotlib inline

num_images = 10

plt.figure(figsize=(15, 3))
for i in range(num_images):
    # RGB image
    plt.subplot(2, num_images, i + 1)
    plt.imshow(X_train_pixel[i])
    plt.axis('off')

    # grayscale image
    plt.subplot(2, num_images, i + num_images + 1)
    plt.imshow(X_training_gray[i], cmap='gray')
    plt.axis('off')

print("RGB along with corresponding grayscaled image\n")
plt.show()

## Feature Extraction (HoGs)

In [ ]:
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

def extract_hog_features(image):
    # Compute HoG features
    hog_features, hog_image = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                                   cells_per_block=cells_per_block, visualize=True)

    # Rescale histogram for better visualization
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

    return hog_features, hog_image_rescaled

In [ ]:
X_train_HoG = []
X_train_HoG_rescaled = []
y_training = np.array(y_train_pixel)

for image in X_training_gray:
    hog_features, hog_image = extract_hog_features(image)
    X_train_HoG.append(hog_features)
    X_train_HoG_rescaled.append(hog_image)

X_training = np.array(X_train_HoG)
X_test_HoG = []
X_test_HoG_rescaled = []
y_testing = np.array(y_test_pixel)

for image in X_testing_gray:
    hog_features, hog_image = extract_hog_features(image)
    X_test_HoG.append(hog_features)
    X_test_HoG_rescaled.append(hog_image)
X_testing = np.array(X_test_HoG)

In [ ]:
np.savez_compressed('X_testing_HoG_.npz', X_testing)
np.savez_compressed('X_training_HoG_.npz', X_training)
np.savez_compressed('y_training_.npz', y_training)
np.savez_compressed('y_testing_.npz', y_testing)

In [ ]:
num_images = 10

plt.figure(figsize=(16, 5))  # Adjust the figure size to accommodate three images per row

for i in range(num_images):
    # RGB image
    plt.subplot(3, num_images, i + 1)
    plt.imshow(X_train_pixel[i])
    plt.title('RGB')
    plt.axis('off')

    # Grayscale image
    plt.subplot(3, num_images, i + 1 + num_images)
    plt.imshow(X_training_gray[i], cmap='gray')
    plt.title('Grayscale')
    plt.axis('off')

    # HoG visualization
    plt.subplot(3, num_images, i + 1 + 2 * num_images)
    # Assuming X_train_HoG_rescaled contains the rescaled HoG images
    plt.imshow(X_train_HoG_rescaled[i], cmap='gray')
    plt.title('HoG')
    plt.axis('off')

plt.tight_layout()  # Adjust spacing between subplots
plt.show()

## Data Statistics

In [ ]:
print( 'X_training shape is {}'.format( X_training.shape) )
print( 'y_training shape is {}'.format( y_training.shape ) )
print( 'X_testing shape is {}'.format( X_testing.shape ) )
print( 'y_testing shape is {}'.format( y_testing.shape ) )

In [ ]:
print( 'Overview of Training Data (Features)')
pd.DataFrame( X_training ).describe()

In [ ]:
import pandas as pd
print( 'Overview of Training Data (Target)')
pd.DataFrame( y_training ).describe()

## Principal Component Analysis

In [ ]:
#Standardizing
X_training_mean = np.mean(X_training, axis=0)  # Compute mean along each feature
X_training_std = np.std(X_training, axis=0)    # Compute standard deviation along each feature
X_training_scaled = (X_training - X_training_mean) / X_training_std

# Standardize the testing data using mean and std computed from training data
X_testing_scaled = (X_testing - X_training_mean) / X_training_std

# After standardization
print('X_training_scaled shape is {}'.format(X_training_scaled.shape))
print('X_testing_scaled shape is {}'.format(X_testing_scaled.shape))

print('X_training_scaled after standardization:')
print(X_training_scaled)

In [ ]:
# Compute the covariance matrix
cov_matrix = np.cov(X_training_scaled, rowvar=False)

# Print the covariance matrix
print("Covariance matrix:")
print(cov_matrix)

In [ ]:
# Find the eigenvectors and eigenvalues of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

# Adjusting the eigenvectors (loadings) that are largest in absolute value to be positive
max_abs_idx = np.argmax(np.abs(eigenvectors), axis=0)
signs = np.sign(eigenvectors[max_abs_idx, range(eigenvectors.shape[0])])
eigenvectors = eigenvectors * signs[np.newaxis, :]
eigenvectors = eigenvectors.T

# Print the eigenvalues and eigenvectors
print("Eigenvalues:")
print(eigenvalues)
print("\nEigenvectors:")
print(eigenvectors)

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eigenvalues[i]), eigenvectors[i, :]) for i in range(len(eigenvalues))]

# Sort the tuples from the highest to the lowest based on eigenvalues magnitude
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# For further usage
eig_vals_sorted = np.array([x[0] for x in eig_pairs])
eig_vecs_sorted = np.array([x[1] for x in eig_pairs])

print(eig_pairs)

In [ ]:
# Calculate explained variance and cumulative explained variance
eig_vals_total = np.sum(eig_vals_sorted)
explained_variance = [(i / eig_vals_total) * 100 for i in eig_vals_sorted]
explained_variance = np.round(explained_variance, 2)
cum_explained_variance = np.cumsum(explained_variance)

print('Explained variance: {}'.format(explained_variance))
print('Cumulative explained variance: {}'.format(cum_explained_variance))


# Plotting cumulative explained variance
num_components = len(explained_variance)
plt.plot(np.arange(1, num_components + 1), cum_explained_variance, '-o')

# Set x-axis tick labels explicitly
custom_ticks = [0, 50, 100, 150, 200, 250, 300]
plt.xticks(custom_ticks)
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.show()

In [ ]:
# Visualizing Over two components
k = 130
W = eig_vecs_sorted[:k, :]  # Projection matrix

X_proj = X_training_scaled.dot(W.T)

print(X_proj.shape)

plt.scatter(X_proj[:, 0], X_proj[:, 1], c=y_training)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('2 components, captures {} of total variation'.format(cum_explained_variance[1]))
plt.show()

In [ ]:
class MyPCA:
    
    def __init__(self, n_components):
        self.n_components = n_components   
        
    def fit(self, X):
        # Standardize data 
        X = X.copy()
        self.mean = np.mean(X, axis=0)
        self.scale = np.std(X, axis=0)
        X_std = (X - self.mean) / self.scale
        
        # Eigendecomposition of covariance matrix       
        cov_mat = np.cov(X_std.T)
        eig_vals, eig_vecs = np.linalg.eig(cov_mat) 
        
        # Adjusting the eigenvectors that are largest in absolute value to be positive    
        max_abs_idx = np.argmax(np.abs(eig_vecs), axis=0)
        signs = np.sign(eig_vecs[max_abs_idx, range(eig_vecs.shape[0])])
        eig_vecs = eig_vecs * signs[np.newaxis, :]
        eig_vecs = eig_vecs.T
       
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[i, :]) for i in range(len(eig_vals))]
        eig_pairs.sort(key=lambda x: x[0], reverse=True)
        eig_vals_sorted = np.array([x[0] for x in eig_pairs])
        eig_vecs_sorted = np.array([x[1] for x in eig_pairs])
        
        self.components = eig_vecs_sorted[:self.n_components, :]
        
        # Explained variance ratio
        self.explained_variance_ratio = [i / np.sum(eig_vals) for i in eig_vals_sorted[:self.n_components]]
        
        self.cum_explained_variance = np.cumsum(self.explained_variance_ratio)

        return self

    def transform(self, X):
        X = X.copy()
        X_std = (X - self.mean) / self.scale
        X_proj = X_std.dot(self.components.T)
        
        return X_proj

In [ ]:
my_pca = MyPCA(n_components=130).fit(X_training)

#print('Components:\n', my_pca.components)
#print('Explained variance ratio:\n', my_pca.explained_variance_ratio)
#print('Cumulative explained variance:\n', my_pca.cum_explained_variance)

X_train_pca = my_pca.transform(X_training)
X_test_pca = my_pca.transform(X_testing)
print('Transformed train data shape:', X_train_pca.shape)
print('Transformed test data shape:', X_test_pca.shape)

## MultiClass Single Layer Perceptron

In [ ]:
# Train perceptron
def initialize_weights(num_features):
    return np.random.uniform(-1, 1, size=num_features + 1)

def perceptron_train(X_train, y_train, iterations, learning_rate, num_classes):
    num_features = X_train.shape[1]
    weights = []

    for class_label in range(num_classes):
        class_weights = initialize_weights(num_features)
        for _ in range(iterations):
            # Shuffle data for each epoch
            shuffled_indices = np.random.permutation(len(X_train))
            X_train_shuffled = X_train[shuffled_indices]
            y_train_shuffled = y_train[shuffled_indices]
            for features, label in zip(X_train_shuffled, y_train_shuffled):
                features_with_bias = np.insert(features, 0, 1)
                target = 1 if label == class_label else 0
                prediction = 1 if np.dot(class_weights, features_with_bias) >= 0 else 0
                error = target - prediction
                class_weights += learning_rate * error * features_with_bias
        weights.append(class_weights)

    return weights

# Training perceptron
weights = perceptron_train(X_training, y_training, iterations=50, learning_rate=0.001, num_classes=10)


In [ ]:
# Make predictions
def predict(X_test, weights):
    predictions = []
    for sample in X_test:
        scores = [np.dot(sample, class_weights[1:]) + class_weights[0] for class_weights in weights]
        predicted_label = np.argmax(scores)
        predictions.append(predicted_label)
    return np.array(predictions)

predicted_labels = predict(X_testing, weights)

# Compute accuracy
accuracy = np.mean(predicted_labels == y_testing)
print("Accuracy:", accuracy)

## K-Nearest Neighbour

In [ ]:
#Calculating Euclidean distance
def euclidean_dist(pointA, pointB):

    distance = np.square(pointA - pointB) # (ai-bi)**2 for every point in the vectors
    distance = np.sum(distance) # adds all values
    distance = np.sqrt(distance)
    return distance

#Calculating distance of test point from all the training points
def distance_from_all_training_points(test_point):
#Returns- dist_array- Array holding distance values for all training data points
    dist_array = np.array([])
    for train_point in X_train_pca:
        dist = euclidean_dist(test_point, train_point)
        dist_array = np.append(dist_array, dist)
    return dist_array

#Implementing KNN Classification model
def KNNClassifier(train_features, train_target, test_features, k = 5):
  predictions = np.array([])
  train_target = train_target.reshape(-1,1)
  for test_point in test_features: # iterating through every test data point
    dist_array = distance_from_all_training_points(test_point).reshape(-1, 1)
    neighbors = np.concatenate((dist_array, train_target), axis=1)
    neighbors_sorted = neighbors[neighbors[:, 0].argsort()] # sorts training points on the basis of distance
    k_neighbors = neighbors_sorted[:k] # selects k-nearest neighbors
    target_class = np.argmax(np.bincount(k_neighbors[:, 1].astype(int)))# selects label with highest frequency
    predictions = np.append(predictions, target_class)
  return predictions

#running inference on test data
test_predictions = KNNClassifier(X_train_pca, y_training, X_test_pca, k = 5)
test_predictions

from sklearn.metrics import classification_report
#Model Evaluation
def accuracy(y_test, y_preds):
    total_correct = sum(1 for true, pred in zip(y_test, y_preds) if true == pred)
    acc = total_correct / len(y_test)
    return acc
acc = accuracy(y_testing, test_predictions)
print('Model accuracy (Scratch) = ', acc*100)
print("Score:\n", classification_report(y_testing, test_predictions))

import seaborn as sns
k_values = list(range(1,20))
accuracy_list = []
for k in k_values:
    test_predictions = KNNClassifier(X_train_pca, y_training, X_test_pca, k)
    accuracy_list.append(accuracy(y_testing, test_predictions))
sns.barplot(k_values, accuracy_list)

## Naive Bayes

There are 10 classes and 130 features in pca reduced dataset. This accounts to more than 1300 likelihood probabilities. Hence, implementing this algorithm manually from scratch is cumbersome. All three Naive Bayes classifier types(Gaussian, Multinomial, Bernoulli) are implemented in the cells below using Sklearn.

In [ ]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
print(X_train_pca.shape)
print(X_test_pca.shape)

In [ ]:
# initializing the classifiers
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()
from sklearn.metrics import accuracy_score
import time

In [ ]:
#Implementing Gaussian Naive Bayes Classifier
try:
  start = time.time()
  clf1.fit(X_train_pca,y_training)
  end = time.time()
  print(f'time taken : {end - start}')

  y_pred1 = clf1.predict(X_test_pca)
  print(y_pred1.shape)
  print(accuracy_score(y_testing,y_pred1))

except Exception as e:
  print(f'Some error occurred : {e}')

In [ ]:
# Implementing Multinomial Naive Bayes Classifier

try:
  start = time.time()
  clf2.fit(X_train_pca,y_training)
  end = time.time()
  print(f'time taken : {end - start}')

  y_pred2 = clf2.predict(X_test_pca)
  print(y_pred2.shape)
  print(accuracy_score(y_testing,y_pred2))

except Exception as e:
  print(f'Some error occurred : {e}')

In [ ]:
negatives = (X_train_pca < 0).sum().sum()

print(f"Total count of negative values : {negatives} Percentage : {(negatives / (50000*130))*100}")

Since a high fraction of values are negative, manipulating this dataset to handle negative values is not a good deal.

In [ ]:
# Implementing Bernoulli Naive Bayes Classifier

try:
  start = time.time()
  clf3.fit(X_train_pca,y_training)
  end = time.time()
  print(f'time taken : {end - start}')

  y_pred3 = clf3.predict(X_test_pca)
  print(y_pred3.shape)
  print(accuracy_score(y_testing,y_pred3))

except Exception as e:
  print(f'Some error occurred : {e}')

### Verdict on Naive Bayes
Given that the dataset is so feature and classes enriched, Multinomial NB could be an appropriate classifier but it does not take negative values which has resulted into an error as stated above. Moreover, there are 10 classes while Bernoulli is used for binary classification. Hence, Naive Bayes is not an ideal classifier for this dataset.